In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

from src.config import load_config, PROJECT_ROOT

app = FastAPI(title="CKD Prediction API", version="1.0")

# Load config, model, preprocessor
cfg = load_config()
model = joblib.load(PROJECT_ROOT / "models/model.pkl")
preprocessor = joblib.load(PROJECT_ROOT / "models/preprocessor.pkl")

numeric_features = cfg["features"]["numeric"]
categorical_features = cfg["features"]["categorical"]


class KidneyInput(BaseModel):
    # numeric features
    Creatinine: float
    BUN: float
    GFR: float
    Urine_Output: float
    Age: float
    Protein_in_Urine: float
    Water_Intake: float

    # categorical features
    Diabetes: str
    Hypertension: str
    Medication: str


def make_feature_array(data: KidneyInput):
    """
    Convert Pydantic model to 2D array in the order expected by preprocessor.
    """
    d = data.dict()
    row = [d[col] for col in numeric_features + categorical_features]
    return np.array([row])


@app.get("/")
def root():
    return {"message": "CKD Prediction API is running."}


@app.post("/predict")
def predict_ckd(features: KidneyInput):
    X = make_feature_array(features)
    X_transformed = preprocessor.transform(X)

    pred = model.predict(X_transformed)[0]
    proba = (
        float(model.predict_proba(X_transformed)[0, 1])
        if hasattr(model, "predict_proba")
        else None
    )

    # Assuming CKD_Status is 1 = CKD, 0 = No CKD
    return {
        "CKD_Status_pred": int(pred),
        "probability_ckd": proba,
    }
